In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('Features.csv',index_col=0)

In [3]:
data.head()

,x1,x2,x3,x4,x5,x6,x7,sentiment
Id,,,,,,,,
0,8.0,14.0,0.0,0.0,0.0,0.0,0.0,1.0
1,13.0,10.0,0.0,0.0,0.0,4.0,1.0,0.0
2,14.0,21.0,0.0,0.0,0.0,4.0,1.0,0.0
3,4.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
4,4.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
data_train = data.drop(['sentiment'], axis = 1, inplace = False)

In [5]:
data_train = data_train.values

In [6]:
data_train = data_train[0:10000, :]

In [7]:
labels = data['sentiment']

In [8]:
labels = labels.values.reshape(labels.shape[0], 1)

In [9]:
labels = labels[0:10000, :]

In [10]:
labels.shape

(10000, 1)

In [11]:
data_train.shape

(10000, 7)

In [32]:
np.random.seed(20)

In [63]:
import numpy as np
np.random.seed(20)
class Neuron:
    def __init__(self, NumberInputs, alpha, isInput):
        # self.W = np.zeros(NumberInputs).reshape(NumberInputs, 1)
        if isInput == False:
            self.W = np.random.rand(NumberInputs).reshape(NumberInputs, 1)
            self.bias = np.random.random(1)
            self.NumberInputs = NumberInputs
            self.alpha = alpha
    

    def sigmoid(self, z):
        return 1/(1 + np.exp(-z))

    def GetOutput(self, X):
        r = np.matmul(self.W.T, X) + self.bias
        return float(self.sigmoid(r))

    def UpdateWeights(self):
        pass



class Layer:
    def __init__(self, NumberNeurons, NumberInputs, alpha, isInput = False):
        self.Neurons = [Neuron(NumberInputs, alpha, isInput) for i in range(NumberNeurons)]
        self.NumberNeurons = NumberNeurons
        self.isInput = isInput

    def LayerOutput(self, X_in):
        Outs = []
        if self.isInput == False:
            for neuron in self.Neurons:
                Outs.append(neuron.GetOutput(X_in))

            return np.array(Outs).reshape(len(Outs), 1)
        else:
            # Ws = np.array([neuron.W for neuron in self.Neurons])
            # print(Ws)
            return X_in


class NeuralNetwork:
    def __init__(self, d, NumIn, alpha = 0.001):
        self.Layers = [0]*len(d)
        self.alpha = alpha
        for key in d:
            if(key == 0):
                self.Layers[key] = Layer(NumberNeurons = d[key], NumberInputs = NumIn, alpha = alpha, isInput=True)
            elif(key > 0):
                self.Layers[key] = Layer(NumberNeurons = d[key], NumberInputs = d[key -1], alpha = alpha, isInput=False)

    def propagate(self, X_in, justAnswer = False):
        X_t = np.array(0)
        L = []
        for index in range(len(self.Layers)):
            if index == 0:
                X_t = self.Layers[index].LayerOutput(X_in.T)
            else:
                X_t = self.Layers[index].LayerOutput(X_t)
            
            L.append(X_t)
        if justAnswer:
            return X_t
        else:
            return (X_t, L)
    
    def backward(self, Outs, y):
        y_hat = Outs[-1]
        deltaB = np.array(y_hat - y).reshape(1,1)
        delta = np.array(y_hat - y).reshape(1,1)*Outs[-2]
        for i in range(len(Outs) - 1):
            index = -1 - i
            X_in = Outs[index -1].copy()
            
            #curr_Wlen = self.Layers[index].Neurons[0].W.shape[0]
            #currW = np.ones(curr_Wlen).reshape(curr_Wlen, 1)
            #for k in range(self.Layers[index+1].NumberNeurons):
            #    currW = np.append(currW, self.Layers[index+1].Neurons[k].W, axis = 1)
            #currW = np.delete(currW, 0, axis = 1)

            #delta = (y-y_hat)*y_hat*(np.ones(y_hat.shape[0]).reshape(y_hat.shape[0],1) - y_hat)
            #deltaW = X_in*delta.T
            #deltaW.shape
            #for j in range(deltaW.shape[1]):
            #    self.Layers[index].Neurons[j].W = self.Layers[index].Neurons[j].W - self.alpha*deltaW[j]
            
            #print(deltaW, deltaW.shape)
            #delta = (y_hat-y)
            if index != -1:
                wlength = self.Layers[index+1].Neurons[0].W.shape[0]
                Ws = np.ones(wlength).reshape(wlength, 1)
                for k in range(self.Layers[index+1].NumberNeurons):
                    Ws = np.append(Ws, self.Layers[index+1].Neurons[k].W, axis = 1)
                Ws = np.delete(Ws, 0, axis = 1)
                #print(Ws, Ws.shape)
                deltaB = np.sum(deltaB.T * Ws, axis = 1).reshape(Ws.shape[0], 1)
                deltaB = deltaB * Outs[index]
                #print(deltaB, deltaB.shape)
                #print(delta.shape, Ws.shape, Outs[index].shape)
                delta = np.sum(delta*Ws*(1 -Outs[index]), axis = 1).reshape(Ws.shape[0],1)
                delta = delta.T*X_in
                #print('DeltaW>\n',delta, delta.shape)
            for j in range(i):
                pass
                #delta = delta*Outs[-2-j]*(1 - Outs[-2-j])
                
                #print('W info>\n',Ws, Ws.shape)
                #deltaB = deltaB*Outs[index]
                #deltaB = delta*Outs[-2-j]*(1 - Outs[-2-j])*W0
            #delta = X_in*delta.T
            #print(delta.shape)
            for j in range(self.Layers[index].NumberNeurons):
                self.Layers[index].Neurons[j].W = self.Layers[index].Neurons[j].W - delta[:,j].reshape(delta.shape[0],1)*self.alpha
                self.Layers[index].Neurons[j].bias = self.Layers[index].Neurons[j].bias - self.alpha*deltaB[j]

    def train(self, X, Y, ephocs = 10):
        rows, cols = X.shape
        for e in range(ephocs):
            error = 0
            for i in range(rows):
                x = X[i,:].reshape(1, cols)
                y = Y[i].reshape(1,1)
                y_hat, Outs = self.propagate(x)
                self.backward(Outs, y)
                error = error + (y_hat - y)*(y_hat - y)
            print(e+1, error)
    
    def predict_one(self, X):
        a = self.propagate(X, justAnswer=True)
        if(a > 0.5):
            a = 1
        else:
            a = 0
        return a
    def predictSet(self, X):
        L = []
        for i in range(X.shape[0]):
            L.append(self.predict_one(X[i,:]))
        return np.array(L).reshape(len(L),1)

#dd = {0:4, 1:4, 2:2,3:1}
#NN = NeuralNetwork(dd, 4, 0.05)
#inp = np.ones(4).reshape(1,4)
#ans = NN.propagate(inp)
#print('Answer:>',ans[0], type(ans[0]), ans[0].shape )
#print(ans[1])
#Y = np.array([0])

#NN.backward(ans[1],Y.reshape(1,1))
#xx = 1


In [64]:
dd = {0: 7, 1:20, 2:1}
NN = NeuralNetwork(dd, 7, 0.0002)
NN.train(data_train, labels, 10)

1 [[3928.40243711]]
2 [[2473.84761994]]
3 [[2455.0133121]]
4 [[2383.45385127]]
5 [[2243.33952261]]
6 [[2097.67553103]]
7 [[1984.67898223]]
8 [[1909.59515266]]
9 [[1856.0106159]]
10 [[1815.13387835]]


In [49]:
dd = {0: 7, 1:20, 2:1}
NN = NeuralNetwork(dd, 7, 0.0002)
NN.train(data_train, labels, 1000)

[[3657.46368265]]
[[2487.55787174]]
[[2476.41628458]]
[[2462.54470421]]
[[2428.79944982]]
[[2334.74018713]]
[[2235.00589967]]
[[2119.92869029]]
[[1991.92343188]]
[[1905.20984952]]
[[1850.22139555]]
[[1810.0696949]]
[[1778.53679043]]
[[1753.07965998]]
[[1732.16014868]]
[[1714.70076519]]
[[1699.93530177]]
[[1687.31995861]]
[[1676.44852434]]
[[1666.99839752]]
[[1658.71373622]]
[[1651.38360152]]
[[1644.82779068]]
[[1638.89456321]]
[[1633.46216375]]
[[1628.4366007]]
[[1623.74350664]]
[[1619.32281968]]
[[1615.12816616]]
[[1611.12664081]]
[[1607.29679617]]
[[1603.62177249]]
[[1600.08320959]]
[[1596.66100326]]
[[1593.33682198]]
[[1590.09695287]]
[[1586.9319698]]
[[1583.83442408]]
[[1580.79726804]]
[[1577.8135853]]
[[1574.87627203]]
[[1571.97749855]]
[[1569.10882952]]
[[1566.26196104]]
[[1563.42941716]]
[[1560.60496119]]
[[1557.78377241]]
[[1554.96243482]]
[[1552.138786]]
[[1549.31171328]]
[[1546.48095584]]
[[1543.64686985]]
[[1540.81007539]]
[[1537.97106417]]
[[1535.13019189]]
[[1532.28877827]

[[1379.26007397]]
[[1379.21194385]]
[[1379.16402415]]
[[1379.11631771]]
[[1379.06882697]]
[[1379.02155404]]
[[1378.97450069]]
[[1378.92766842]]
[[1378.88105851]]
[[1378.83467198]]
[[1378.78850967]]
[[1378.74257223]]
[[1378.69686015]]
[[1378.65137377]]
[[1378.6061133]]
[[1378.56107882]]
[[1378.51627029]]
[[1378.47168759]]
[[1378.42733049]]
[[1378.38319867]]
[[1378.33929176]]
[[1378.29560927]]
[[1378.25215069]]
[[1378.20891543]]
[[1378.16590283]]
[[1378.12311219]]
[[1378.08054277]]
[[1378.03819379]]
[[1377.9960644]]
[[1377.95415374]]
[[1377.91246091]]
[[1377.87098497]]
[[1377.82972497]]
[[1377.78867992]]
[[1377.7478488]]
[[1377.70723058]]
[[1377.66682421]]
[[1377.62662862]]
[[1377.58664272]]
[[1377.54686541]]
[[1377.50729558]]
[[1377.4679321]]
[[1377.42877383]]
[[1377.38981964]]
[[1377.35106836]]
[[1377.31251884]]
[[1377.27416992]]
[[1377.23602043]]
[[1377.1980692]]
[[1377.16031505]]
[[1377.1227568]]
[[1377.08539328]]
[[1377.04822331]]
[[1377.01124572]]
[[1376.97445932]]
[[1376.93786295]

[[1370.62506799]]
[[1370.62062921]]
[[1370.61621897]]
[[1370.6118372]]
[[1370.60748386]]
[[1370.60315889]]
[[1370.59886222]]
[[1370.59459382]]
[[1370.59035363]]
[[1370.58614159]]
[[1370.58195766]]
[[1370.57780179]]
[[1370.57367394]]
[[1370.56957406]]
[[1370.56550212]]
[[1370.56145806]]
[[1370.55744184]]
[[1370.55345344]]
[[1370.54949281]]
[[1370.54555992]]
[[1370.54165473]]
[[1370.53777721]]
[[1370.53392732]]
[[1370.53010504]]
[[1370.52631034]]
[[1370.52254318]]
[[1370.51880354]]
[[1370.51509139]]
[[1370.51140672]]
[[1370.50774948]]
[[1370.50411967]]
[[1370.50051726]]
[[1370.49694222]]
[[1370.49339454]]
[[1370.4898742]]
[[1370.48638118]]
[[1370.48291547]]
[[1370.47947704]]
[[1370.47606588]]
[[1370.47268198]]
[[1370.46932532]]
[[1370.46599588]]
[[1370.46269367]]
[[1370.45941866]]
[[1370.45617084]]
[[1370.45295021]]
[[1370.44975675]]
[[1370.44659046]]
[[1370.44345133]]
[[1370.44033934]]
[[1370.43725449]]
[[1370.43419678]]
[[1370.4311662]]
[[1370.42816274]]
[[1370.4251864]]
[[1370.4222371

In [65]:
ans = NN.predictSet(data_train)

In [66]:
ans

array([[0],
       [1],
       [0],
       ...,
       [0],
       [1],
       [1]])

In [67]:
labels

array([[1.],
       [0.],
       [0.],
       ...,
       [0.],
       [1.],
       [1.]])

In [68]:
def score(y1, y2):
    d = y1 - y2
    d = d*d
    d = np.sum(d)
    t_error = d
    score = t_error/y1.shape[0]
    return score, t_error

In [69]:
score(ans, labels)

(0.2432, 2432.0)

In [56]:
ans.shape

(10000, 1)

In [70]:
sum(labels)

array([4965.])